In [4]:
# from data import info_url,posts_url,reels_url,hashtags_url
import pandas as pd
import requests

In [ ]:
# collected data from different API calls
info_data = requests.get(info, headers=headers, params=querystring).json()
posts_data = requests.get(post, headers=headers, params=querystring).json()
reels_data = requests.get(reels, headers=headers, params=querystring).json()
hashtags_data = requests.get(
    hastag, headers=headers, params=querystring).json()

# Converting the collected data into DataFrames
info_df = pd.DataFrame(info_data['data'])
posts_df = pd.DataFrame(posts_data['data'])
reels_df = pd.DataFrame(reels_data['data'])
hashtags_df = pd.DataFrame(hashtags_data['data'])

# Save the datasets if needed
info_df.to_csv("info_data.csv", index=False)
posts_df.to_csv("posts_data.csv", index=False)
reels_df.to_csv("reels_data.csv", index=False)
hashtags_df.to_csv("hashtags_data.csv", index=False)

In [ ]:
# Remove duplicates
posts_df.drop_duplicates(inplace=True)
reels_df.drop_duplicates(inplace=True)
hashtags_df.drop_duplicates(inplace=True)

In [ ]:
# Fill missing values or remove rows with missing data
posts_df.fillna('', inplace=True)
reels_df.fillna('', inplace=True)
hashtags_df.fillna('', inplace=True)

In [ ]:
import re


def clean_text(text):
    # Removing URLs and special characters
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = text.lower()
    return text


posts_df['caption'] = posts_df['caption'].apply(clean_text)
reels_df['caption'] = reels_df['caption'].apply(clean_text)
hashtags_df['hashtags'] = hashtags_df['hashtags'].apply(clean_text)

In [ ]:
# Merging all datasets into a single DataFrame
final_data = pd.merge(posts_df[['caption', 'url']], reels_df[[
                      'caption', 'url']], on='url', how='outer')
final_data = pd.merge(
    final_data, hashtags_df[['hashtags', 'url']], on='url', how='outer')

# Combine captions and hashtags for a complete content-based recommendation
final_data['content'] = final_data['caption'] + " " + final_data['hashtags']

# Drop any remaining duplicates or nulls
final_data.drop_duplicates(subset='url', inplace=True)
final_data.dropna(subset=['content'], inplace=True)

In [ ]:
# optional because hamne pahle hi nikala hai
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()


def get_sentiment(text):
    score = analyzer.polarity_scores(text)
    return score['compound']


final_data['sentiment'] = final_data['content'].apply(get_sentiment)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorizing the text content
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(final_data['content'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# Create a function to recommend based on content
def get_recommendations(index, cosine_sim=cosine_sim):
    # Get the pairwise similarity scores of all content
    sim_scores = list(enumerate(cosine_sim[index]))

    # Sort content based on similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top 10 most similar content
    sim_scores = sim_scores[1:11]

    # Return the URLs of the most similar content
    content_indices = [i[0] for i in sim_scores]
    return final_data['url'].iloc[content_indices]

In [ ]:
# Test the recommendation system for the first post
print(get_recommendations(0))

In [ ]:
import pickle

# Save the TF-IDF vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# Save the cosine similarity matrix
with open('cosine_similarity.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)

In [54]:
import pandas as pd
from faker import Faker
import random
import numpy as np

In [15]:

# Initialize Faker
fake = Faker()

# Function to create synthetic dataset


def create_synthetic_data(num_samples=1000):
    data = {
        'user_id': [],
        'post_id': [],
        'Username': [],
        'Caption': [],
        'Hashtags': [],
        'Likes': [],
        'Comments': []
    }

    for _ in range(num_samples):
        # Generate user data
        user_id = random.randint(0, 1000)
        post_id = random.randint(0, 1000)
        username = fake.user_name()
        caption = fake.sentence(nb_words=random.randint(5, 15))
        hashtags = ' '.join([fake.word() for _ in range(random.randint(1, 5))])
        likes = random.randint(0, 1000)
        comments = random.randint(0, 200)

        # Append data
        data['user_id'].append(user_id)
        data['post_id'].append(user_id)
        data['Username'].append(username)
        data['Caption'].append(caption)
        data['Hashtags'].append(hashtags)
        data['Likes'].append(likes)
        data['Comments'].append(comments)

    return pd.DataFrame(data)


# Create synthetic dataset
synthetic_data = create_synthetic_data(num_samples=1000)

# Save to CSV
synthetic_data.to_csv('sentiments.csv', index=False)
print(synthetic_data.head())

,post_id,user_id,caption,hashtags,likes,comments,timestamp
0,1,15,Exploring the mountains,#sunset,339,61,2024-01-01 00:00:00
1,2,3,Coffee time,#morning,706,68,2024-01-01 01:00:00
2,3,11,Love this beach,#blessed,317,41,2024-01-01 02:00:00
3,4,14,City lights,#travel,58,89,2024-01-01 03:00:00
4,5,18,Road trip,#blessed,720,57,2024-01-01 04:00:00


In [ ]:
# Create synthetic dataset
synthetic_data = create_synthetic_data(num_samples=1000)

# Save to CSV
synthetic_data.to_csv('sentiments.csv', index=False)
print(synthetic_data.head())

In [2]:
import pandas as pd
df = pd.read_csv('sentiments.csv')
df.head(1000)

,post_id,user_id,caption,hashtags,likes,comments,share,timestamp
0,1,2,Healthy breakfast,#healthy,542,24,89,2024-01-01 00:00:00
1,2,2,Throwback to summer,#summer,595,25,106,2024-01-01 01:00:00
2,3,14,Road trip,#roadtrip,676,21,102,2024-01-01 02:00:00
3,4,17,Exploring the mountains,#hiking,460,47,89,2024-01-01 03:00:00
4,5,9,Family time,#qualitytime,360,31,33,2024-01-01 04:00:00
...,...,...,...,...,...,...,...,...
995,996,19,Enjoying the sunset,#sunset,895,43,98,2024-02-11 11:00:00
996,997,15,Healthy living,#fitness,363,10,35,2024-02-11 12:00:00
997,998,13,Coffee time,#caffeine,645,13,98,2024-02-11 13:00:00
998,999,8,Healthy breakfast,#food,337,43,46,2024-02-11 14:00:00


In [ ]:
# Import necessary libraries
import pandas as pd
from textblob import TextBlob

# Step 1: Load the CSV file
df = pd.read_csv('sentiments.csv')

# Step 2: Data Preprocessing
# Keep only relevant columns (Caption, Hashtags)
df = df[['caption', 'hashtags']]

# Remove any missing or NaN values
df.dropna(subset=['caption', 'hashtags'], inplace=True)

# Step 3: Sentiment Analysis Function


def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity  # Returns a score between -1 and 1


# Step 4: Calculate Sentiment Scores for Caption and Hashtags
df['caption_score'] = df['Caption'].apply(analyze_sentiment)
df['hashtag_score'] = df['Hashtags'].apply(analyze_sentiment)

# Step 5: Combine Scores to Determine Overall Sentiment
df['overall_score'] = (df['caption_score'] + df['hashtag_score']) / 2

# Step 6: Categorize Sentiment


def categorize_sentiment(score):
    if score < 0:
        return 'Negative'
    elif score > 0:
        return 'Positive'
    else:
        return 'Neutral'


df['sentiment'] = df['overall_score'].apply(categorize_sentiment)

# Step 7: Create a new DataFrame with required columns
final_df = df[['Caption', 'Hashtags', 'sentiment', 'overall_score']]

# Step 8: Save to a new CSV file
final_df.to_csv('instagram_reach_with_sentiments.csv', index=False)

print("Sentiment analysis complete! New file saved as 'instagram_sentiments.csv'.")

In [3]:
df2 = pd.read_csv("instagram_reach_with_sentiments.csv")
df2

,caption,hashtags,sentiment,overall_score
0,Healthy breakfast,#healthy,Positive,0.50
1,Throwback to summer,#summer,Neutral,0.00
2,Road trip,#roadtrip,Neutral,0.00
3,Exploring the mountains,#hiking,Neutral,0.00
4,Family time,#qualitytime,Neutral,0.00
...,...,...,...,...
995,Enjoying the sunset,#sunset,Positive,0.25
996,Healthy living,#fitness,Positive,0.25
997,Coffee time,#caffeine,Neutral,0.00
998,Healthy breakfast,#food,Positive,0.25


In [6]:
df2.columns

Index(['caption', 'hashtags', 'sentiment', 'overall_score'], dtype='object')

In [1]:
# Import necessary libraries
import pandas as pd
from textblob import TextBlob

In [2]:
df = pd.read_csv('instagram_reach_with_sentiments.csv')
df.nunique()

Caption          1000
Hashtags          982
sentiment           3
overall_score     332
dtype: int64

In [19]:
df['sentiment'].value_counts()

sentiment
Positive    520
Neutral     248
Negative    232
Name: count, dtype: int64

In [9]:
# Import necessary libraries
import pandas as pd
from textblob import TextBlob
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
# Using Random Forest for better accuracy
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# Step 1: Load the CSV file
df = pd.read_csv('sentiments.csv')

# Step 2: Data Preprocessing
# Keep only relevant columns (Caption, Hashtags)
df = df[['caption', 'hashtags']]

# Remove any missing or NaN values
df.dropna(subset=['caption', 'hashtags'], inplace=True)


def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity


# Step 4: Calculate Sentiment Scores for Caption and Hashtags
df['caption_score'] = df['caption'].apply(analyze_sentiment)
df['hashtag_score'] = df['hashtags'].apply(analyze_sentiment)

# Step 5: Combine Scores to Determine Overall Sentiment
df['overall_score'] = (df['caption_score'] + df['hashtag_score']) / 2

# Step 6: Categorize Sentiment


def categorize_sentiment(score):
    if score < 0:
        return 'Negative'
    elif score > 0:
        return 'Positive'
    else:
        return 'Neutral'


df['sentiment'] = df['overall_score'].apply(categorize_sentiment)

# Step 7: Create a new DataFrame with required columns
final_df = df[['caption', 'hashtags', 'sentiment', 'overall_score']]

# Step 8: Save to a new CSV file
final_df.to_csv('instagram_reach_with_sentiments.csv', index=False)

print("Sentiment analysis complete! New file saved as 'instagram_reach_with_sentiments.csv'.")

# Load the new CSV file for training the model
df = pd.read_csv('instagram_reach_with_sentiments.csv')

# Prepare Data for Machine Learning
X = df['caption'] + " " + df['hashtags']  # Features (text data)
y = df['sentiment']  # Labels (sentiment categories)

# Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Convert Text Data into Numerical Features
tfidf = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Use Random Over Sampling to Balance Classes
ros = RandomOverSampler(random_state=42)
X_train_balanced, y_train_balanced = ros.fit_resample(X_train_tfidf, y_train)

print("Original training set shape:", Counter(y_train))
print("Balanced training set shape:", Counter(y_train_balanced))

# Train a Machine Learning Model (Random Forest Classifier)
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train_balanced, y_train_balanced)

# Save the model and the TF-IDF vectorizer
with open('sentiment_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(tfidf, vectorizer_file)

# Evaluate the Model
y_pred = model.predict(X_test_tfidf)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Make Predictions on New Data
def predict_sentiment(new_caption, new_hashtags):
    # Load the model and vectorizer
    with open('sentiment_model.pkl', 'rb') as model_file:
        loaded_model = pickle.load(model_file)

    with open('tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
        loaded_vectorizer = pickle.load(vectorizer_file)

    new_text = new_caption + " " + new_hashtags
    new_tfidf = loaded_vectorizer.transform([new_text])
    return loaded_model.predict(new_tfidf)[0]

Sentiment analysis complete! New file saved as 'instagram_reach_with_sentiments.csv'.
Original training set shape: Counter({'Neutral': 453, 'Negative': 189, 'Positive': 158})
Balanced training set shape: Counter({'Neutral': 453, 'Positive': 453, 'Negative': 453})
Classification Report:
              precision    recall  f1-score   support

    Negative       1.00      1.00      1.00        48
     Neutral       1.00      1.00      1.00       113
    Positive       1.00      1.00      1.00        39

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [10]:
import pickle
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the Random Forest model
with open('sentiment_model.pkl', 'rb') as random_forest_model_file:
    random_forest_model = pickle.load(random_forest_model_file)

# Load the TF-IDF vectorizer
with open('tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    tfidf_vectorizer = pickle.load(vectorizer_file)

# # Define the test data
# test_data = pd.DataFrame(
#     {'text': ['I love this product', 'This product is terrible']})

# give as user input 

# Preprocess the text data


def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text


# Apply preprocessing
test_data['text'] = test_data['text'].apply(preprocess_text)

# Vectorize the preprocessed text data
X_test = tfidf_vectorizer.transform(test_data['text'])

# Make predictions
predictions = random_forest_model.predict(X_test)

# Print the results
for text, prediction in zip(test_data['text'], predictions):
    print(f"Text: '{text}' - Sentiment: {prediction}")

Text: 'i love this product' - Sentiment: Positive
Text: 'this product is terrible' - Sentiment: Neutral


In [ ]:
import pickle
import pandas as pd
import re
from flask import Flask, request, render_template, jsonify

app = Flask(__name__)

# Load the Random Forest model and TF-IDF vectorizer
with open('sentiment_model.pkl', 'rb') as model_file:
    random_forest_model = pickle.load(model_file)

with open('tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    tfidf_vectorizer = pickle.load(vectorizer_file)

# Preprocess the text data


def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text


@app.route('/')
def index():
    return render_template('index.html')  # Render the input form


@app.route('/predict', methods=['POST'])
def predict():
    user_input = request.form['text']  # Get the user input from the form
    # Create a DataFrame for the input
    test_data = pd.DataFrame({'text': [user_input]})

    # Apply preprocessing
    test_data['text'] = test_data['text'].apply(preprocess_text)

    # Vectorize the preprocessed text data
    X_test = tfidf_vectorizer.transform(test_data['text'])

    # Make predictions
    predictions = random_forest_model.predict(X_test)

    # Return the result
    return jsonify({'text': user_input, 'sentiment': predictions[0]})


if __name__ == "__main__":
    app.run(debug=True)

In [77]:
df2 = pd.read_csv('instagram_reach_with_sentiments.csv')
df2

,caption,hashtags,sentiment,overall_score
0,Healthy breakfast,#healthy,Positive,0.50
1,Throwback to summer,#summer,Neutral,0.00
2,Road trip,#roadtrip,Neutral,0.00
3,Exploring the mountains,#hiking,Neutral,0.00
4,Family time,#qualitytime,Neutral,0.00
...,...,...,...,...
995,Enjoying the sunset,#sunset,Positive,0.25
996,Healthy living,#fitness,Positive,0.25
997,Coffee time,#caffeine,Neutral,0.00
998,Healthy breakfast,#food,Positive,0.25


In [80]:
df2['sentiment'].value_counts()

sentiment
Neutral     566
Negative    237
Positive    197
Name: count, dtype: int64

In [81]:
new_caption = "I am sad"
new_hashtags = "#sad"
predicted_sentiment = predict_sentiment(new_caption, new_hashtags)
print(f"Predicted Sentiment: {predicted_sentiment}")

Predicted Sentiment: Negative


In [46]:
import pandas as pd
import random
import numpy as np
# Random seed for reproducibility
np.random.seed(90)

In [65]:
# Sample data lists for captions and hashtags (with logical associations)
caption_hashtag_pairs = {
    "Enjoying the sunset": ["#sunset", "#evening", "#nature"],
    "Morning vibes": ["#morning", "#freshstart", "#sunrise"],
    "Love this beach": ["#beach", "#ocean", "#sun"],
    "Hiking adventures": ["#hiking", "#outdoors", "#adventure"],
    "Coffee time": ["#coffee", "#morning", "#caffeine"],
    "Feeling blessed": ["#blessed", "#grateful", "#thankful"],
    "Weekend getaway": ["#weekend", "#travel", "#getaway"],
    "Road trip": ["#roadtrip", "#adventure", "#travel"],
    "Exploring the mountains": ["#mountains", "#hiking", "#nature"],
    "City lights": ["#city", "#nightlife", "#urban"],
    "Healthy living": ["#healthyliving", "#fitness", "#wellness"],
    "Family time": ["#family", "#love", "#qualitytime"],
    "Throwback to summer": ["#summer", "#memories", "#throwback"],
    "Dinner with friends": ["#dinner", "#friends", "#goodtimes"],
    "Workout motivation": ["#workout", "#fitness", "#motivation"],
    "Chasing waterfalls": ["#waterfalls", "#nature", "#hiking"],
    "Sunset views": ["#sunset", "#views", "#sky"],
    "Beach day": ["#beach", "#sun", "#sand"],
    "Healthy breakfast": ["#breakfast", "#healthy", "#food"],
    "Travel goals": ["#travel", "#adventure", "#explore"],
    "I am feeling very sad":["#sad","#low","#feel"],
    "it makes me sad":["#depression","#stress","#trauma"],
    "the sad fact is that he's lost his touch" : ["#lost","#touch","#sad"],
    "they looked at her with sad, anxious faces" : ["#anxious","#sad","#faces"],
    "I am feeling very happy":["#happy","#joy","#feel"],
    "the sad fact is that he's lost his touch" : ["#lost","#touch","#sad"],
    "they looked at her with sad, anxious faces" : ["#anxious","#sad","#faces"],
    "a sad day for us all":["#sad","#day","#all"],
    "sad to say, she never lived to see it":["#lived","sad"]
}

In [66]:
# Simulate users with varying engagement patterns
users = {
    1: {'activity_level': 'high', 'avg_likes': 800, 'avg_comments': 50},
    2: {'activity_level': 'medium', 'avg_likes': 500, 'avg_comments': 30},
    3: {'activity_level': 'low', 'avg_likes': 200, 'avg_comments': 10},
    4: {'activity_level': 'high', 'avg_likes': 700, 'avg_comments': 40},
    5: {'activity_level': 'medium', 'avg_likes': 400, 'avg_comments': 25},
    6: {'activity_level': 'low', 'avg_likes': 150, 'avg_comments': 5},
    7: {'activity_level': 'high', 'avg_likes': 900, 'avg_comments': 60},
    8: {'activity_level': 'medium', 'avg_likes': 450, 'avg_comments': 20},
    9: {'activity_level': 'low', 'avg_likes': 180, 'avg_comments': 8},
    10: {'activity_level': 'high', 'avg_likes': 750, 'avg_comments': 45},
}

In [67]:
# Expand user pool up to 20 with varied activity levels
for user_id in range(4, 21):
    users[user_id] = {
        'activity_level': random.choice(['high', 'medium', 'low']),
        'avg_likes': random.randint(200, 800),
        'avg_comments': random.randint(10, 50),
        
    }

In [68]:
# Number of posts
n = 1000

# Generate random posts with relationships between columns
data = {
    'post_id': [i for i in range(1, n+1)],
    # Random user IDs from 1 to 20
    'user_id': [random.randint(1, 20) for _ in range(n)],
}

In [69]:
# Generate realistic captions, hashtags, likes, comments, and timestamps
captions = []
hashtags = []
likes = []
comments = []
share = []
timestamps = pd.date_range(start='2024-01-01', periods=n, freq='H')

In [70]:
for i in range(n):
    user_id = data['user_id'][i]

    # Choose a random caption and corresponding hashtag
    caption = random.choice(list(caption_hashtag_pairs.keys()))
    hashtags_list = caption_hashtag_pairs[caption]
    hashtag = random.choice(hashtags_list)

    # Likes and comments depend on user's activity level
    user_profile = users[user_id]
    base_likes = user_profile['avg_likes']
    base_comments = user_profile['avg_comments']

    # Add some variability (more likes and comments for certain captions or time of day)
    # Evening posts get more likes
    engagement_factor = 1.5 if "sunset" in caption or timestamps[i].hour >= 18 else 1
    post_likes = int(base_likes * engagement_factor *
                     random.uniform(0.8, 1.2))  # Add randomness
    share_count = int(post_likes * random.uniform(0.05, 0.2))
    post_comments = int(base_comments * engagement_factor *
                        random.uniform(0.8, 1.2))
    

    # Append to lists
    captions.append(caption)
    hashtags.append(hashtag)
    likes.append(post_likes)
    share.append(share_count)
    comments.append(post_comments)

In [71]:
# Add to the data dictionary
data['caption'] = captions
data['hashtags'] = hashtags
data['likes'] = likes
data['comments'] = comments
data['share'] = share
data['timestamp'] = timestamps

# Create DataFrame
instagram_df = pd.DataFrame(data)

# Display the first few rows of the dataset
instagram_df.head()

,post_id,user_id,caption,hashtags,likes,comments,share,timestamp
0,1,2,Healthy breakfast,#healthy,542,24,89,2024-01-01 00:00:00
1,2,2,Throwback to summer,#summer,595,25,106,2024-01-01 01:00:00
2,3,14,Road trip,#roadtrip,676,21,102,2024-01-01 02:00:00
3,4,17,Exploring the mountains,#hiking,460,47,89,2024-01-01 03:00:00
4,5,9,Family time,#qualitytime,360,31,33,2024-01-01 04:00:00


In [72]:
instagram_df['caption'].value_counts()

caption
a sad day for us all                          46
Sunset views                                  46
it makes me sad                               45
Coffee time                                   43
Travel goals                                  43
I am feeling very sad                         42
Hiking adventures                             41
Feeling blessed                               41
Dinner with friends                           41
the sad fact is that he's lost his touch      39
Healthy breakfast                             38
Love this beach                               38
Road trip                                     38
sad to say, she never lived to see it         37
Enjoying the sunset                           37
Healthy living                                37
Family time                                   36
Workout motivation                            35
Morning vibes                                 33
I am feeling very happy                       32
Chasing wate

In [73]:
instagram_df.size

8000

In [74]:
instagram_df.to_csv('sentiments.csv', index=False)

In [75]:
df = pd.read_csv('sentiments.csv')
df

,post_id,user_id,caption,hashtags,likes,comments,share,timestamp
0,1,2,Healthy breakfast,#healthy,542,24,89,2024-01-01 00:00:00
1,2,2,Throwback to summer,#summer,595,25,106,2024-01-01 01:00:00
2,3,14,Road trip,#roadtrip,676,21,102,2024-01-01 02:00:00
3,4,17,Exploring the mountains,#hiking,460,47,89,2024-01-01 03:00:00
4,5,9,Family time,#qualitytime,360,31,33,2024-01-01 04:00:00
...,...,...,...,...,...,...,...,...
995,996,19,Enjoying the sunset,#sunset,895,43,98,2024-02-11 11:00:00
996,997,15,Healthy living,#fitness,363,10,35,2024-02-11 12:00:00
997,998,13,Coffee time,#caffeine,645,13,98,2024-02-11 13:00:00
998,999,8,Healthy breakfast,#food,337,43,46,2024-02-11 14:00:00
